<a href="https://colab.research.google.com/github/papichoolo/healthchatbot/blob/main/Medllama_use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers gradio peft bitsandbytes

In [2]:
PEFT_MODEL = "tmberooney/medllama"

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from transformers import LlamaForCausalLM, LlamaTokenizer

from threading import Thread

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("tmberooney/medllama")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",load_in_4bit=True, device_map="auto")
model = PeftModel.from_pretrained(model, "tmberooney/medllama")
tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = model.to('cuda:0')



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [6]:
tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

In [7]:
generation_config = model.generation_config
generation_config.max_new_tokens = 500
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [8]:
%%time
device = "cuda:0"

prompt = """
[INST]<human>: Paracetamol doesnt work in reducing my fever what should i Do?[/INST]
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )
final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
final_outputs = cut_off_text(final_outputs, '\n<human>: ')
final_outputs = remove_substring(final_outputs, prompt)

print(final_outputs)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


 Sorry to hear that Paracetamol is not working for you. Here are some other options you can consider:

1. Take anti-inflammatory medication: If your fever is caused by inflammation, taking anti-inflammatory medication such as ibuprofen or aspirin can help reduce the fever.
2. Try acetaminophen: If Paracetamol is not working, you can try taking acetaminophen. It is a pain reliever and fever reducer that can help reduce your fever.
3. Use a cold compress: Applying a cold compress to your forehead or the back of your neck can help reduce your fever.
4. Take a warm bath: Taking a warm bath can help reduce your fever.
5. Get plenty of rest: Getting plenty of rest can help your body fight off the underlying infection that is causing your fever.
6. Drink plenty of fluids: Drinking plenty of fluids can help replace the fluids lost due to fever and sweating.
7. Try herbal remedies: Some herbal remedies such as ginger, turmeric, and ginkgo biloba can help reduce fever.
8. Consult a doctor: If yo